# Welcome to TapeAgents!


**TapeAgents** is a framework that leverages a structured, replayable log (**Tape**) of the agent session to facilitate all stages of the LLM Agent development lifecycle. In TapeAgents, the agent reasons by processing the tape and the LLM output to produce new thoughts, actions, control flow steps and append them to the tape. The environment then reacts to the agent’s actions by likewise appending observation steps to the tape.

In [18]:
from tapeagents.llms import OpenrouterLLM

api_key = "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"
llm = OpenrouterLLM(
    model_name="meta-llama/llama-3.3-70b-instruct:free",  # https://openrouter.ai/meta-llama/llama-3.3-70b-instruct:free
    api_token=api_key,
    parameters={"temperature": 0.1},
)

In [20]:
llm.quick_response("How are you?")

"I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to assist you with any questions or tasks you may have! How about you? How's your day going so far?"

## Create environment with the set of tools that the agent can use

In [ ]:
import os

from tapeagents.environment import ToolCollectionEnvironment
from tapeagents.tools.calculator import Calculator
from tapeagents.tools.simple_browser import SimpleBrowser
from tapeagents.tools.web_search import WebSearch

# key for the web search api, you can get a free key at https://serper.dev/
os.environ["SERPER_API_KEY"] = "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"

environment = ToolCollectionEnvironment(tools=[WebSearch(), SimpleBrowser(), Calculator()])
environment.initialize()
print(environment.tools_description())

ModuleNotFoundError: No module named 'docling'

## Simple agent that think once and makes one tool call before anwering

In [4]:
from tapeagents.agent import Agent
from tapeagents.nodes import StandardNode, Stop

system_prompt = "You are an assistant that can use web search, browser and calculator. Be brief and concise."
agent = Agent.create(
    llms=llm,
    nodes=[
        # nodes are steps that are executed in order,
        # each node can asks llm to produce a response or call some tool from the environment,
        # when all nodes are executed, the agent will stop
        StandardNode(
            name="think",
            system_prompt=system_prompt,
            guidance="let's think how to use available tools to answer the user's question, do not answer the question yet",
        ),
        StandardNode(
            name="act",
            system_prompt=system_prompt,
            guidance="call the proposed tool",
            use_known_actions=True,  # this will use all the actions defined in the environment
            use_function_calls=True,  # this will allow to use model tool calling capabilities
        ),
        StandardNode(
            name="answer",
            system_prompt=system_prompt,
            guidance="give a final answer to the user's question based on the tool result",
        ),
        Stop(),
    ],
    known_actions=environment.actions(),
    tools_description=environment.tools_description(),
)

In [5]:
from tapeagents.dialog_tape import DialogTape, UserStep
from tapeagents.orchestrator import main_loop

user_question = "What is the closest distance from the Earth to the Moon?"
tape = DialogTape(steps=[UserStep(content=user_question)])

for event in main_loop(agent, tape, environment):
    if event.agent_event and event.agent_event.step:
        step = event.agent_event.step
        print(f"Agent step ({step.metadata.node}):\n{step.llm_view()}\n---")
    elif event.observation:
        print(f"Tool call result:\n{event.observation.short_view()}\n---")
    elif event.agent_tape:
        tape = event.agent_tape

print("Agent final answer:", tape[-2].reasoning)  # last step before Stop node

Agent step (think):
To find the closest distance from the Earth to the Moon, we can:

1. Use a web search to find information on the Moon's orbit and its closest point to Earth.
2. Utilize a calculator to perform any necessary calculations, such as converting units or calculating averages.
3. Browse online resources, such as NASA's website or astronomy websites, for accurate and up-to-date information on the Earth-Moon distance.

Let's proceed with a web search to gather more information.
---
Agent step (act):
{
  "kind": "search_action",
  "source": "web",
  "query": "closest distance from Earth to Moon",
  "time_interval": ""
}
---
Tool call result:
{
  "kind": "search_results_observation",
  "query": "closest distance from Earth to Moon",
  "serp": [
    {
      "title": "How far is the moon from Earth? - Space",
      "url": "https://www.space.com/18145-how-far-is-the-moon.html",
      "snippet": "When the moon is at its farthest distance from Earth, known as apogee, it is about 25

## Agent that can think and act in a loop

In [6]:
from tapeagents.core import FinalStep

react_agent = Agent.create(
    llms=llm,
    nodes=[
        StandardNode(
            name="plan",
            system_prompt=system_prompt,
            guidance="write a concise multi-step plan to find the answer for the given question. Do not answer the question yet",
        ),
        StandardNode(
            name="reflect",
            system_prompt=system_prompt,
            guidance="evaluate current state of the solution, propose the next tool to call to move forward. If the plan is finished, call FinalStep to answer the question",
        ),
        StandardNode(
            name="act",
            system_prompt=system_prompt,
            guidance="produce proposed tool call",
            use_known_actions=True,
            use_function_calls=True,
            steps=FinalStep,  # if this step is produced, the agent will stop
            next_node="reflect",  # here we make a loop back to the reflect node
        ),
    ],
    known_actions=environment.actions(),
    tools_description=environment.tools_description(),
)

In [7]:
# question from the GAIA benchmark validation set https://huggingface.co/datasets/gaia-benchmark/GAIA
user_question2 = "If Eliud Kipchoge could maintain his record-making marathon pace indefinitely, how many hours would it take him to run the distance between the Earth and the Moon its closest approach?"
tape = DialogTape(steps=[UserStep(content=user_question2)])

for event in main_loop(react_agent, tape, environment):
    if event.agent_event and event.agent_event.step:
        step = event.agent_event.step
        print(f"Agent step ({step.metadata.node}):\n{step.llm_view()}\n---")
    elif event.observation:
        print(f"Tool call result:\n{event.observation.short_view()}\n---")
    elif event.agent_tape:
        tape = event.agent_tape

print("\n\nAgent final answer:", tape[-1].reason)

Agent step (plan):
To find the answer, follow these steps:

1. Find Eliud Kipchoge's record-making marathon pace (distance per hour).
2. Determine the distance between the Earth and the Moon at its closest approach.
3. Calculate the time it would take to cover this distance at Kipchoge's pace (time = distance / pace).
4. Convert the result to hours.
---
Agent step (reflect):
Current state: We have a plan to solve the problem, but we don't have the necessary values for Eliud Kipchoge's record-making marathon pace and the distance between the Earth and the Moon at its closest approach.

Next tool to call: Web search to find the values for Eliud Kipchoge's record-making marathon pace and the distance between the Earth and the Moon at its closest approach.
---
Agent step (act):
{
  "kind": "search_action",
  "source": "web",
  "query": "Eliud Kipchoge marathon pace",
  "time_interval": ""
}
---
Agent step (act):
{
  "kind": "search_action",
  "source": "web",
  "query": "distance between E